In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.eps'
if not 'location' in locals():
    #save location.  First one is for running on home PC, second for running on the work laptop.  May need to make a global change
    location = 'E:\\Documents\\Dan\\Code\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'
    #location = 'C:\\Users\\dhendrickson\\Documents\\github\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'


#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)

In [2]:
#Project Specific packages:
import zipfile
import DWFT as fp
import os as os
import pywt as py
import pandas as pd
my_cmap = plt.get_cmap('gray')

In [3]:
def PlotFingerPrint(data, xName, title='', FP='mexh'):
    
    FpScat = fp.getLabeledThumbprint(data, FP)
    FpScat = np.matrix(FpScat)
    
    Al,Ms  = np.meshgrid(xName,np.linspace(1,50,50))

    fig1 = plt.figure(figsize=(6.67,3.75))
    ax1 = plt.axes()
    cs1 = ax1.contourf(Al,Ms, FpScat,cmap=my_cmap,levels=10)

    if Titles: plt.title(title)
    if Saving: plt.savefig(location+title.replace(" ", "").replace(":", "").replace(",", "").replace(".txt","")+FFormat)

    plt.show()

Directory = "E:\\Documents\\Dan\\PhD\\Play\\ASC\\"
files = os.listdir(Directory)
Dots = 1000
Start = 2615

for Filename in files:
    print(Filename[-3:])
    if Filename[-3:] == 'csv':
        Results = []
        DataSet = np.genfromtxt(open(Directory+'/'+Filename,'r'), delimiter=',',skip_header=4)
        #plt.plot(DataSet[:,0],DataSet[:,1], label = 'X')
        #plt.plot(DataSet[:,0],DataSet[:,2], label = 'Y')
        for j in range(Dots):
            i = j + Start
            Results.append([DataSet[i,1],np.sqrt(DataSet[i,2]**2+DataSet[i,3]**2+DataSet[i,4]**2)])
        Results = np.matrix(Results)
        plt.plot(Results[:,0],Results[:,1], label = 'R')

        title = Filename + 'R-'
        if Ledgends: plt.legend()
        if Titles: plt.title(title)
        if Saving: plt.savefig(location+title.replace(" ", "").replace(":", "").replace(",", "")+FFormat)
        plt.show()
        PlotFingerPrint(np.ravel(Results[:,1]),Results[:,0],title)

In [4]:
#DataSet = np.genfromtxt(open("E:\\Documents\\Dan\\PhD\\Play\\ASC\\60kPoints-210709-1026.csv",'r'), delimiter=',',skip_header=4)
#DataSet = np.genfromtxt(open("C:\\Users\\dhendrickson\\Pone Drive\\OneDrive - The Port of Virginia\\Shared with Everyone\\60kAccel-210713-1700.csv",'r'), delimiter=',',skip_header=4)
folder = "C:\\Users\\dhendrickson\\Desktop\\AccelData\\test\\"

Results = []
for i in range(np.shape(DataSet)[0]):
    c = DataSet[i,1]
    x = DataSet[i,2]
    y = DataSet[i,3]
    z = DataSet[i,4]
    r = np.sqrt(x**2 + y**2 + z**2)
    t = np.arccos(z / r)
    p = np.arctan(y / x)
    Results.append([c,x,y,z,r,t,p])
Results = np.matrix(Results)

Names = ['Time',
         'x',
         'y',
         'z',
         'r',
         'theta',
         'phi']

StdDev = [  np.std(Results[:,1]),
            np.std(Results[:,2]),
            np.std(Results[:,3]),
            np.std(Results[:,4]),
            np.std(Results[:,5]),
            np.std(Results[:,6])]

BlockSize = 500

for i in range(int(np.shape(DataSet)[0]/BlockSize)):
    Start = i * BlockSize
    End = Start + BlockSize

    LocalStdDev = [ np.std(Results[Start:End,1]),
                    np.std(Results[Start:End,2]),
                    np.std(Results[Start:End,3]),
                    np.std(Results[Start:End,4]),
                    np.std(Results[Start:End,5]),
                    np.std(Results[Start:End,6])]
    
    for k in range(6):
        
        if np.abs((LocalStdDev[k])/StdDev[k]) > 2:
            plt.plot(Results[Start:End,0],Results[Start:End,k], label = Names[k])

            title = Names[k] + ' starting at step ' + str(i * BlockSize)
            if Ledgends: plt.legend()
            if Titles: plt.title(title)
            if Saving: plt.savefig(location+title.replace(" ", "").replace(":", "").replace(",", "")+FFormat)
            plt.show()
            PlotFingerPrint(np.ravel(Results[Start:End,k]),Results[Start:End,0],title)


In [14]:
files = os.listdir(folder)
Results = []
for Filename in files:
    if Filename[-3:] == 'csv':
         DataSet = np.genfromtxt(open(folder+Filename,'r'), delimiter=',',skip_header=4)
         for i in range(3):
            fprint = fp.getLabeledThumbprint(DataSet[:,i+2], 'mexh')
            avgRidge = float(sum(fp.RidgeCount(fprint[:,500:59500])))/59000.0
            Results.append([Filename[11:13], Filename[13:15],Filename[16:18],Filename[17:19], i+2, avgRidge ])
print(Results)

[['08', '01', '06', '61', 1, 0.01265], ['08', '01', '06', '61', 2, 0.0267], ['08', '01', '06', '61', 3, 0.7719666666666667]]


In [12]:
df = pd.DataFrame(data=Results.astype('float'))
df.to_csv(folder+'-output.csv', sep=',', header=False, float_format='%.5f')

08


(50, 59000)
